In [1]:
#drive mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
#ir a la carpeta ct_triage
!ls /content/drive/MyDrive/ct_triage

brain_stroke_dataset.tar.gz  ct_triage	ct_triage_data	runs


In [3]:
#verificar git status
%cd /content/drive/MyDrive/ct_triage/ct_triage
!git status

/content/drive/MyDrive/ct_triage/ct_triage
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   src/dataset.py
	modified:   src/models.py
	modified:   src/train_baseline.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	notebooks/ct_triage.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [4]:
#instalar requirements
%pip install -r requirements.txt

In [5]:
#IMPORTS
import sys
from pathlib import Path
import pandas as pd
import os
import importlib
import torch


In [6]:
#mas imports
import src.dataset
from src.models import make_resnet18
from src.models import make_backbone
from src.train_baseline import train_baseline

In [7]:
#establecer root
ROOT = Path("/content/drive/MyDrive/ct_triage/ct_triage").resolve()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

print("ROOT =", ROOT)


ROOT = /content/drive/MyDrive/ct_triage/ct_triage


In [9]:
#crear tar para cargar datos
%cd /content/drive/MyDrive/ct_triage
!tar -czf brain_stroke_dataset.tar.gz -C ct_triage_data Brain_Stroke_CT_Dataset


Se truncaron las últimas líneas 5000 del resultado de transmisión.
tar: Brain_Stroke_CT_Dataset/Normal/PNG/16046.png: file changed as we read it
tar: Brain_Stroke_CT_Dataset/Normal/PNG/16120.png: file changed as we read it
tar: Brain_Stroke_CT_Dataset/Normal/PNG/16075.png: file changed as we read it
tar: Brain_Stroke_CT_Dataset/Normal/PNG/16050.png: file changed as we read it
tar: Brain_Stroke_CT_Dataset/Normal/PNG/15999.png: file changed as we read it
tar: Brain_Stroke_CT_Dataset/Normal/PNG/16054.png: file changed as we read it
tar: Brain_Stroke_CT_Dataset/Normal/PNG/16170.png: file changed as we read it
tar: Brain_Stroke_CT_Dataset/Normal/PNG/16060.png: file changed as we read it
tar: Brain_Stroke_CT_Dataset/Normal/PNG/16074.png: file changed as we read it
tar: Brain_Stroke_CT_Dataset/Normal/PNG/16171.png: file changed as we read it
tar: Brain_Stroke_CT_Dataset/Normal/PNG/16123.png: file changed as we read it
tar: Brain_Stroke_CT_Dataset/Normal/PNG/16077.png: file changed as we read 

In [10]:
# Crear carpeta destino local
!mkdir -p /content/data_local

# Ruta del .tar.gz en Drive
TAR_PATH="/content/drive/MyDrive/ct_triage/brain_stroke_dataset.tar.gz"

# Extraer en /content/data_local
!tar -xzf "$TAR_PATH" -C /content/data_local


In [11]:
DATA_ROOT = "/content/data_local/Brain_Stroke_CT_Dataset"
rows = []

label_map = {
    "Normal": 0,
    "Ischemia": 1,
    "Bleeding": 1,
}

# -------------------------------
# 1) NORMAL / ISCHEMIA / BLEEDING
# -------------------------------
for cls, label in label_map.items():
    png_dir = os.path.join(DATA_ROOT, cls, "PNG")
    mask_dir = os.path.join(DATA_ROOT, cls, "MASKS")

    if not os.path.isdir(png_dir):
        continue

    for fname in os.listdir(png_dir):
        if fname.lower().endswith(".png"):
            img_path = os.path.join(png_dir, fname)
            mask_path = os.path.join(mask_dir, fname)
            rows.append([img_path, mask_path, label])

# -------------------------------
# 2) EXTERNAL_TEST use labels.csv
# -------------------------------
labels_csv = os.path.join(DATA_ROOT, "External_Test", "labels.csv")
png_dir = os.path.join(DATA_ROOT, "External_Test", "PNG")
mask_dir = os.path.join(DATA_ROOT, "External_Test", "MASKS")

if os.path.isfile(labels_csv):
    df_labels = pd.read_csv(labels_csv)

    for _, r in df_labels.iterrows():
        image_id = str(r["image_id"])
        label = int(r["Stroke"])

        fname = f"{image_id}.png"
        img_path = os.path.join(png_dir, fname)
        mask_path = os.path.join(mask_dir, fname)

        rows.append([img_path, mask_path, label])

# -------------------------------
# BUILD FINAL DF
# -------------------------------
df = pd.DataFrame(rows, columns=["img_path", "mask_path", "label"])

df_path = "/content/data_local/full_dataset.csv"
df.to_csv(df_path, index=False)

df.head(), df.columns



(                                            img_path  \
 0  /content/data_local/Brain_Stroke_CT_Dataset/No...   
 1  /content/data_local/Brain_Stroke_CT_Dataset/No...   
 2  /content/data_local/Brain_Stroke_CT_Dataset/No...   
 3  /content/data_local/Brain_Stroke_CT_Dataset/No...   
 4  /content/data_local/Brain_Stroke_CT_Dataset/No...   
 
                                            mask_path  label  
 0  /content/data_local/Brain_Stroke_CT_Dataset/No...      0  
 1  /content/data_local/Brain_Stroke_CT_Dataset/No...      0  
 2  /content/data_local/Brain_Stroke_CT_Dataset/No...      0  
 3  /content/data_local/Brain_Stroke_CT_Dataset/No...      0  
 4  /content/data_local/Brain_Stroke_CT_Dataset/No...      0  ,
 Index(['img_path', 'mask_path', 'label'], dtype='object'))

In [15]:
from src.train_baseline import train_baseline

In [17]:
#resnet18
cfg_resnet18 = {
    "df": df,
    "data_root": "/content/data_local/Brain_Stroke_CT_Dataset",
    "img_size": 224,
    "batch_size": 8,
    "use_masks": False,
    "roi_mode": "keep",
    "num_workers": 2,
    "val_frac": 0.2,
    "lr": 1e-4,
    "weight_decay": 1e-4,
    "epochs": 10,
    "seed": 42,
    "out_dir": "runs/baseline_resnet18",
    "arch": "resnet18",
}


results_resnet18 = train_baseline(cfg_resnet18)


[1/10] loss=0.3374 val_f1=0.8746 val_auc=0.9676 (45.1s)


[2/10] loss=0.1742 val_f1=0.9043 val_auc=0.9884 (41.1s)


[3/10] loss=0.1047 val_f1=0.9480 val_auc=0.9935 (41.1s)


[4/10] loss=0.0760 val_f1=0.9278 val_auc=0.9901 (41.2s)


[5/10] loss=0.0538 val_f1=0.9243 val_auc=0.9863 (40.8s)


[6/10] loss=0.0451 val_f1=0.9409 val_auc=0.9922 (41.3s)


[7/10] loss=0.0266 val_f1=0.9533 val_auc=0.9911 (42.9s)


[8/10] loss=0.0600 val_f1=0.9243 val_auc=0.9868 (40.7s)


[9/10] loss=0.0316 val_f1=0.9518 val_auc=0.9929 (39.2s)


[10/10] loss=0.0224 val_f1=0.9681 val_auc=0.9945 (40.7s)


In [19]:
import torch
from src.models import make_backbone
device = "cuda" if torch.cuda.is_available() else "cpu"

best_ckpt_path = results_resnet18["best_ckpt"]
print("Best checkpoint ResNet-18:", best_ckpt_path)

ckpt = torch.load(best_ckpt_path, map_location=device, weights_only=False)

model_resnet18 = make_backbone(
    arch="resnet18",
    num_classes=2,
    pretrained=False
)
model_resnet18.load_state_dict(ckpt["model"])
model_resnet18.to(device)
model_resnet18.eval()

Best checkpoint ResNet-18: runs/baseline_resnet18/best_f1_epoch10_f10.9681.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [20]:
#resnet50
cfg_resnet50 = {
    "df": df,
    "data_root": "/content/data_local/Brain_Stroke_CT_Dataset",
    "img_size": 224,
    "batch_size": 8,
    "use_masks": False,
    "roi_mode": "keep",
    "num_workers": 2,
    "val_frac": 0.2,
    "lr": 1e-4,
    "weight_decay": 1e-4,
    "epochs": 10,
    "seed": 42,
    "out_dir": "runs/baseline_resnet50",
    "arch": "resnet50",
}


results_resnet50 = train_baseline(cfg_resnet50)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 146MB/s]


[1/10] loss=0.3347 val_f1=0.8765 val_auc=0.9675 (78.8s)


[2/10] loss=0.1571 val_f1=0.9394 val_auc=0.9877 (79.8s)


[3/10] loss=0.0885 val_f1=0.9275 val_auc=0.9890 (79.8s)


...

[6/10] loss=0.0382 val_f1=0.9492 val_auc=0.9880 (79.4s)


[7/10] loss=0.0372 val_f1=0.9475 val_auc=0.9890 (78.2s)


[8/10] loss=0.0285 val_f1=0.9426 val_auc=0.9884 (79.7s)


[9/10] loss=0.0160 val_f1=0.9492 val_auc=0.9922 (79.7s)


[10/10] loss=0.0214 val_f1=0.9534 val_auc=0.9903 (78.1s)


In [22]:
import torch
from src.models import make_backbone
device = "cuda" if torch.cuda.is_available() else "cpu"

# results_resnet50 = train_baseline(cfg_resnet50)
best_ckpt_path = results_resnet50["best_ckpt"]
print("Best checkpoint ResNet-50:", best_ckpt_path)

ckpt = torch.load(best_ckpt_path, map_location=device, weights_only=False)

model_resnet50 = make_backbone(
    arch="resnet50",
    num_classes=2,
    pretrained=False
)
model_resnet50.load_state_dict(ckpt["model"])
model_resnet50.to(device)
model_resnet50.eval()

Best checkpoint ResNet-50: runs/baseline_resnet50/best_f1_epoch10_f10.9534.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [23]:
#densenet121
cfg_densenet121 = {
    "df": df,
    "data_root": "/content/data_local/Brain_Stroke_CT_Dataset",
    "img_size": 224,
    "batch_size": 8,
    "use_masks": False,
    "roi_mode": "keep",
    "num_workers": 2,
    "val_frac": 0.2,
    "lr": 1e-4,
    "weight_decay": 1e-4,
    "epochs": 10,
    "seed": 42,
    "out_dir": "runs/baseline_densenet121",
    "arch": "densenet121",
}


results_densenet121 = train_baseline(cfg_densenet121)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


100%|██████████| 30.8M/30.8M [00:00<00:00, 219MB/s]


[1/10] loss=0.3292 val_f1=0.8428 val_auc=0.9793 (90.0s)


[2/10] loss=0.1943 val_f1=0.9266 val_auc=0.9879 (88.3s)


[3/10] loss=0.1321 val_f1=0.9206 val_auc=0.9861 (88.6s)


...

[6/10] loss=0.0532 val_f1=0.9396 val_auc=0.9900 (105.3s)


[7/10] loss=0.0743 val_f1=0.9579 val_auc=0.9915 (88.6s)


[8/10] loss=0.0239 val_f1=0.9354 val_auc=0.9902 (91.3s)


[9/10] loss=0.0470 val_f1=0.9440 val_auc=0.9890 (87.6s)


[10/10] loss=0.0302 val_f1=0.9667 val_auc=0.9871 (89.7s)


In [25]:
import torch
from src.models import make_backbone
device = "cuda" if torch.cuda.is_available() else "cpu"

# results_densenet121 = train_baseline(cfg_densenet121)
best_ckpt_path = results_densenet121["best_ckpt"]
print("Best checkpoint DenseNet-121:", best_ckpt_path)

ckpt = torch.load(best_ckpt_path, map_location=device, weights_only=False)

model_densenet121 = make_backbone(
    arch="densenet121",
    num_classes=2,
    pretrained=False
)
model_densenet121.load_state_dict(ckpt["model"])
model_densenet121.to(device)
model_densenet121.eval()


Best checkpoint DenseNet-121: runs/baseline_densenet121/best_f1_epoch10_f10.9667.pth


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [27]:
#efficientnet
cfg_efficientnet = {
    "df": df,
    "data_root": "/content/data_local/Brain_Stroke_CT_Dataset",
    "img_size": 224,
    "batch_size": 8,
    "use_masks": False,
    "roi_mode": "keep",
    "num_workers": 2,
    "val_frac": 0.2,
    "lr": 1e-4,
    "weight_decay": 1e-4,
    "epochs": 10,
    "seed": 42,
    "out_dir": "runs/baseline_efficientnet",
    "arch": "efficientnet_b0",
}


results_efficientnet = train_baseline(cfg_efficientnet)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 146MB/s]


[1/10] loss=0.3662 val_f1=0.8644 val_auc=0.9649 (54.2s)


[2/10] loss=0.2091 val_f1=0.9172 val_auc=0.9814 (55.8s)


[3/10] loss=0.1392 val_f1=0.9384 val_auc=0.9877 (54.5s)


...

[6/10] loss=0.0546 val_f1=0.9455 val_auc=0.9936 (54.9s)


[7/10] loss=0.0537 val_f1=0.9556 val_auc=0.9927 (55.7s)


[8/10] loss=0.0436 val_f1=0.9635 val_auc=0.9914 (55.7s)


[9/10] loss=0.0187 val_f1=0.9382 val_auc=0.9908 (55.5s)


[10/10] loss=0.0254 val_f1=0.9349 val_auc=0.9918 (56.2s)


In [28]:
import torch
from src.models import make_backbone
device = "cuda" if torch.cuda.is_available() else "cpu"

# results_efficientnet = train_baseline(cfg_efficientnet)
best_ckpt_path = results_efficientnet["best_ckpt"]
print("Best checkpoint EfficientNet-B0:", best_ckpt_path)

ckpt = torch.load(best_ckpt_path, map_location=device, weights_only=False)

model_effnet_b0 = make_backbone(
    arch="efficientnet_b0",
    num_classes=2,
    pretrained=False
)
model_effnet_b0.load_state_dict(ckpt["model"])
model_effnet_b0.to(device)
model_effnet_b0.eval()


Best checkpoint EfficientNet-B0: runs/baseline_efficientnet/best_f1_epoch8_f10.9635.pth


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [29]:
# Filtrar solo imágenes de External_Test
df_ext = df[df["img_path"].str.contains("External_Test")].reset_index(drop=True)
print("Filas en External_Test:", len(df_ext))
df_ext.head()


Filas en External_Test: 200


,img_path,mask_path,label
0,/content/data_local/Brain_Stroke_CT_Dataset/Ex...,/content/data_local/Brain_Stroke_CT_Dataset/Ex...,0
1,/content/data_local/Brain_Stroke_CT_Dataset/Ex...,/content/data_local/Brain_Stroke_CT_Dataset/Ex...,0
2,/content/data_local/Brain_Stroke_CT_Dataset/Ex...,/content/data_local/Brain_Stroke_CT_Dataset/Ex...,0
3,/content/data_local/Brain_Stroke_CT_Dataset/Ex...,/content/data_local/Brain_Stroke_CT_Dataset/Ex...,0
4,/content/data_local/Brain_Stroke_CT_Dataset/Ex...,/content/data_local/Brain_Stroke_CT_Dataset/Ex...,0


In [31]:
#nuevo
import torch
from src.models import make_backbone

device = "cuda" if torch.cuda.is_available() else "cpu"

def load_model_from_results(results):
    """
    Carga el modelo correcto (arquitectura + pesos)
    a partir del dict `results` devuelto por train_baseline.
    """
    best_ckpt_path = results["best_ckpt"]
    print("Cargando checkpoint:", best_ckpt_path)

    ckpt = torch.load(best_ckpt_path, map_location=device, weights_only=False)
    cfg_trained = ckpt["cfg"]
    arch = cfg_trained.get("arch", "resnet18")  # default si no estaba

    print("Arquitectura entrenada:", arch)

    model = make_backbone(
        arch=arch,
        num_classes=2,
        pretrained=False
    )
    model.load_state_dict(ckpt["model"])
    model.to(device)
    model.eval()
    return model


In [32]:
#nuevo
from torch.utils.data import DataLoader
from src.dataset import ExternalBinaryDS
from src.utils import eval_epoch

def eval_on_external(model, df_ext, img_size=224, use_masks=False, roi_mode="keep"):
    ds_ext = ExternalBinaryDS(
        df=df_ext,
        data_root="",         # no importa, los paths ya son absolutos
        size=img_size,
        use_masks=use_masks,
        roi_mode=roi_mode
    )

    dl_ext = DataLoader(ds_ext, batch_size=8, shuffle=False)
    metrics = eval_epoch(model, dl_ext, device=device, idx_normal=0)
    return metrics


In [34]:
#nuevo
# ResNet-18
model_resnet18 = load_model_from_results(results_resnet18)
metrics_resnet18 = eval_on_external(model_resnet18, df_ext)
print("ResNet-18:", metrics_resnet18)

# ResNet-50
model_resnet50 = load_model_from_results(results_resnet50)
metrics_resnet50 = eval_on_external(model_resnet50, df_ext)
print("ResNet-50:", metrics_resnet50)

# DenseNet-121
model_densenet121 = load_model_from_results(results_densenet121)
metrics_densenet121 = eval_on_external(model_densenet121, df_ext)
print("DenseNet-121:", metrics_densenet121)

# EfficientNet-B0
model_effnet_b0 = load_model_from_results(results_efficientnet)
metrics_effnet_b0 = eval_on_external(model_effnet_b0, df_ext)
print("EfficientNet-B0:", metrics_effnet_b0)


Cargando checkpoint: runs/baseline_resnet18/best_f1_epoch10_f10.9681.pth
Arquitectura entrenada: resnet18
ResNet-18: {'acc': 0.99, 'prec': 1.0, 'rec': 0.9714285714285714, 'f1': 0.9855072463768116, 'auc': np.float64(0.9954945054945056), 'pos_rate_pred': 0.34}
Cargando checkpoint: runs/baseline_resnet50/best_f1_epoch10_f10.9534.pth
Arquitectura entrenada: resnet50
ResNet-50: {'acc': 0.99, 'prec': 0.9857142857142858, 'rec': 0.9857142857142858, 'f1': 0.9857142857142858, 'auc': np.float64(0.9879670329670329), 'pos_rate_pred': 0.35}
Cargando checkpoint: runs/baseline_densenet121/best_f1_epoch10_f10.9667.pth
Arquitectura entrenada: densenet121
DenseNet-121: {'acc': 0.985, 'prec': 1.0, 'rec': 0.9571428571428572, 'f1': 0.9781021897810219, 'auc': np.float64(0.9934065934065934), 'pos_rate_pred': 0.335}
Cargando checkpoint: runs/baseline_efficientnet/best_f1_epoch8_f10.9635.pth
Arquitectura entrenada: efficientnet_b0
EfficientNet-B0: {'acc': 0.985, 'prec': 1.0, 'rec': 0.9571428571428572, 'f1': 0.9

In [35]:
#nuevo
import pandas as pd

rows = [
    {"model": "resnet18",    **metrics_resnet18},
    {"model": "resnet50",    **metrics_resnet50},
    {"model": "densenet121", **metrics_densenet121},
    {"model": "efficientnet_b0", **metrics_effnet_b0},
]

metrics_df = pd.DataFrame(rows)
metrics_df


,model,acc,prec,rec,f1,auc,pos_rate_pred
0,resnet18,0.990,1.000000,0.971429,0.985507,0.995495,0.340
1,resnet50,0.990,0.985714,0.985714,0.985714,0.987967,0.350
2,densenet121,0.985,1.000000,0.957143,0.978102,0.993407,0.335
3,efficientnet_b0,0.985,1.000000,0.957143,0.978102,0.997802,0.335


In [37]:
#nuevo
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

def confusion_on_external(model, df_ext, img_size=224, use_masks=False, roi_mode="keep"):
    ds_ext = ExternalBinaryDS(
        df=df_ext,
        data_root="",
        size=img_size,
        use_masks=use_masks,
        roi_mode=roi_mode
    )
    dl_ext = DataLoader(ds_ext, batch_size=8, shuffle=False)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.eval()
    ys_true, ys_pred = [], []

    with torch.no_grad():
        for xb, yb, _ in dl_ext:
            xb = xb.to(device)
            logits = model(xb)
            probs = torch.softmax(logits, dim=1).cpu().numpy()
            yhat = (1.0 - probs[:, 0] >= 0.5).astype(int)  # 0=Normal, 1=Stroke
            ys_true += yb.numpy().tolist()
            ys_pred += yhat.tolist()

    cm = confusion_matrix(ys_true, ys_pred)
    print("Matriz de confusión (filas=verdadero, columnas=predicho):")
    print(cm)
    print("\nReporte de clasificación:")
    print(classification_report(ys_true, ys_pred, target_names=["Normal", "Stroke"]))


confusion_on_external(model_resnet50, df_ext)


Matriz de confusión (filas=verdadero, columnas=predicho):
[[129   1]
 [  1  69]]

Reporte de clasificación:
              precision    recall  f1-score   support

      Normal       0.99      0.99      0.99       130
      Stroke       0.99      0.99      0.99        70

    accuracy                           0.99       200
   macro avg       0.99      0.99      0.99       200
weighted avg       0.99      0.99      0.99       200

